In [1]:
import pandas as pd 
import numpy as np 
import os 
import re
import sys 
import ast 

In [2]:
results_dir = "../../european-city-data/rag-sustainability/results/results-combined_prompts/"
folders = os.listdir(results_dir)
results_df = pd.read_csv(f"{results_dir}/recommended_cities_gemma.csv")

cities_df = pd.read_csv("../../european-city-data/cities/worldcities.csv")
cities = list(cities_df['city'])

eucities_df = pd.read_csv("../../european-city-data/archive/city_abstracts_embeddings.csv")
eu_cities = list(cities_df['city'])

sar_results_dir = "../../european-city-data/rag-sustainability/results/results-combined_prompts_SAR/"
sar_results = pd.read_csv(f"{sar_results_dir}/recommended_cities_sar_gemma.csv")

In [3]:
results_df.head()

,model,prompt_id,rec_cities,response
0,gemma2,prompt_17_gemini-ui,"['Gaziantep', 'Konya']",It seems you've provided information about two...
1,gemma2,prompt_27_gemini-1.5-pro-001,"['Coimbra', 'Varna']",It looks like you've provided information abou...
2,gemma2,prompt_42_gpt-4o-mini,['Amsterdam'],The provided text gives a list of events happe...
3,gemma2,prompt_0_gpt-4o-mini,['Strasbourg'],It seems like you're providing text snippets a...
4,gemma2,prompt_44_gemini-1.5-pro-001,['Prague'],It looks like you've provided information abou...


In [4]:
sar_results.head()

,model,prompt_id,rec_cities_sar,response_sar
0,gemma2,prompt_17_gemini-ui,"['Gaziantep', 'Konya']",The text you provided gives information about ...
1,gemma2,prompt_27_gemini-1.5-pro-001,"['Coimbra', 'Varna']",Let's break down how to structure this informa...
2,gemma2,prompt_42_gpt-4o-mini,['Amsterdam'],The provided text outlines a list of events ha...
3,gemma2,prompt_0_gpt-4o-mini,['Strasbourg'],Let's break down how to structure this informa...
4,gemma2,prompt_44_gemini-1.5-pro-001,['Prague'],This data looks like a collection of tourist a...


In [5]:
combined_df = pd.merge(
    left=results_df, 
    right=sar_results,
    how='left',
    left_on=['model', 'prompt_id'],
    right_on=['model', 'prompt_id'],
)
combined_df.head()

,model,prompt_id,rec_cities,response,rec_cities_sar,response_sar
0,gemma2,prompt_17_gemini-ui,"['Gaziantep', 'Konya']",It seems you've provided information about two...,"['Gaziantep', 'Konya']",The text you provided gives information about ...
1,gemma2,prompt_27_gemini-1.5-pro-001,"['Coimbra', 'Varna']",It looks like you've provided information abou...,"['Coimbra', 'Varna']",Let's break down how to structure this informa...
2,gemma2,prompt_42_gpt-4o-mini,['Amsterdam'],The provided text gives a list of events happe...,['Amsterdam'],The provided text outlines a list of events ha...
3,gemma2,prompt_0_gpt-4o-mini,['Strasbourg'],It seems like you're providing text snippets a...,['Strasbourg'],Let's break down how to structure this informa...
4,gemma2,prompt_44_gemini-1.5-pro-001,['Prague'],It looks like you've provided information abou...,['Prague'],This data looks like a collection of tourist a...


In [6]:
llama = combined_df[combined_df['model'] == 'llama3point1-instruct']
mistral = combined_df[combined_df['model'] == 'mistral-instruct']
gemma = combined_df[combined_df['model'] == 'gemma2']

### Do both models recommend the same city? In how many cases do they differ?

In [12]:
sort_cities = lambda x: sorted(ast.literal_eval(x))

df = combined_df.copy(deep=True)
print(df.columns)
df['rec_cities'] = df['rec_cities'].apply(sort_cities)
df['rec_cities_sar'] = df['rec_cities_sar'].apply(sort_cities)
def all_values_same(l1, l2, l3):
    return all(x == y == z for x, y, z in zip(l1, l2, l3))

def count_same_response(df):
    count_same_values = 0

    for i, row in df.iterrows():
        l1 = sorted(row['llama3point1-instruct'])
        l2 = sorted(row['mistral-instruct'])
        l3 = sorted(row['gemma2'])

        # if len(l1) == len(l2) and l1 == l2:
        if all_values_same(l1, l2, l3):
            count_same_values += 1


    # print("Count of records where values of C are the same across different values of A:\n", count_same_values)
    return count_same_values


# count_rec = count_same_response(df, 'rec_cities')
# count_rec

Index(['model', 'prompt_id', 'rec_cities', 'response', 'rec_cities_sar',
       'response_sar'],
      dtype='object')


In [13]:
pivot_table = df.pivot_table(index='prompt_id', columns='model', values='rec_cities', aggfunc=lambda x: x)
pivot_table

model,gemma2,llama3point1-instruct,mistral-instruct
prompt_id,,,
prompt_0_gemini-1.5-pro-001,[Nalchik],[Innsbruck],[]
prompt_0_gemini-ui,[Nalchik],[Innsbruck],"[Bergen, Kars, Kaunas, Nalchik, Plovdiv]"
prompt_0_gpt-4o-mini,[Strasbourg],"[Kaunas, Strasbourg, Thessaloniki]",[Sibiu]
prompt_10_gemini-1.5-pro-001,[Kazan],[Sarajevo],[]
prompt_10_gemini-ui,"[London, Mykolaiv]",[London],[Antalya]
...,...,...,...
prompt_8_gemini-ui,[Barcelona],[Milan],[Barcelona]
prompt_8_gpt-4o-mini,[Plovdiv],[Bergen],[]
prompt_9_gemini-1.5-pro-001,[Ankara],"[Budapest, Luxembourg, Mykolaiv, Nicosia, Orad...","[Amsterdam, Ankara, Budapest]"


In [15]:
pivot_table_sus = df.pivot_table(index='prompt_id', columns='model', values='rec_cities_sar', aggfunc=lambda x: x)
pivot_table_sus

model,gemma2,llama3point1-instruct,mistral-instruct
prompt_id,,,
prompt_0_gemini-1.5-pro-001,[Nalchik],"[Bergen, Innsbruck, Kars, Kaunas, Nalchik, Plo...","[Kars, Kaunas, Nalchik, Plovdiv]"
prompt_0_gemini-ui,[Nalchik],"[Innsbruck, Plovdiv, Stavanger]","[Bergen, Kars, Kaunas, Nalchik, Plovdiv]"
prompt_0_gpt-4o-mini,[Strasbourg],"[Kaunas, Oradea, Sibiu]",[Strasbourg]
prompt_10_gemini-1.5-pro-001,[Kazan],"[Erzurum, Sarajevo, Vitoria-Gasteiz]","[Aalborg, Kazan, London, Nalchik]"
prompt_10_gemini-ui,"[London, Mykolaiv]",[Varna],"[Mykolaiv, Simferopol]"
...,...,...,...
prompt_8_gemini-ui,"[Antalya, Barcelona]",[Milan],"[Barcelona, Braga, Milan, Tallinn]"
prompt_8_gpt-4o-mini,[Plovdiv],"[Arkhangelsk, Bergen, Varna]",[]
prompt_9_gemini-1.5-pro-001,[Ankara],[Budapest],"[Amsterdam, Ankara, Vienna]"


In [16]:
count_non_sus = count_same_response(pivot_table)

In [17]:
count_sus = count_same_response(pivot_table_sus)

In [18]:
count_non_sus

50

In [19]:
count_sus

44

The models rarely agree with their responses. Providing sustainability information does increase the model agreement, albeit marginally, perhaps because of the additional constraint placed on the models to consider the S-Fairness score. WHY??

### How many prompt responses have atleast one city in common between Llama and Mistral?

In [24]:
def has_common_element(l1, l2, l3):
    return bool(set(l1) & set(l2) & set(l3))
def count_atleast_one_common_city(df):
    count_same_values = 0

    for i, row in df.iterrows():
        l1 = set(row['llama3point1-instruct'])
        l2 = set(row['mistral-instruct'])
        l3 = set (row["gemma2"])

        if len(l1 & l2 & l3):
        # if has_common_element(l1, l2, l3):
            count_same_values += 1


    # print("Count of records where values of C are the same across different values of A:\n", count_same_values)
    return count_same_values

In [25]:
one_common = count_atleast_one_common_city(pivot_table)
one_common_sus = count_atleast_one_common_city(pivot_table_sus)

print(one_common, one_common_sus)

27 29


Checking if both models predict atleast one common city fares much better, with both models agreeing approximately 33% of the time

## With SAR

In [9]:
llama = combined_df[combined_df['model'] == 'llama3point1-instruct']
mistral = combined_df[combined_df['model'] == 'mistral-instruct']

In [10]:
sort_cities = lambda x: sorted(ast.literal_eval(x))

df = combined_df.copy(deep=True)
df['rec_cities'] = df['rec_cities'].apply(sort_cities)
df['rec_cities_sar'] = df['rec_cities_sar'].apply(sort_cities)

def count_same_response(df):
    count_same_values = 0

    for i, row in df.iterrows():
        l1 = sorted(row['llama3point1-instruct'])
        l2 = sorted(row['mistral-instruct'])

        if len(l1) == len(l2) and l1 == l2:
            count_same_values += 1


    # print("Count of records where values of C are the same across different values of A:\n", count_same_values)
    return count_same_values


In [29]:
pivot_table = df.pivot_table(index='prompt_id', columns='model', values='rec_cities', aggfunc=lambda x: x)
pivot_table

model,gemma2,llama3point1-instruct,mistral-instruct
prompt_id,,,
prompt_0_gemini-1.5-pro-001,[Nalchik],[Innsbruck],[]
prompt_0_gemini-ui,[Nalchik],[Innsbruck],"[Bergen, Kars, Kaunas, Nalchik, Plovdiv]"
prompt_0_gpt-4o-mini,[Strasbourg],"[Kaunas, Strasbourg, Thessaloniki]",[Sibiu]
prompt_10_gemini-1.5-pro-001,[Kazan],[Sarajevo],[]
prompt_10_gemini-ui,"[London, Mykolaiv]",[London],[Antalya]
...,...,...,...
prompt_8_gemini-ui,[Barcelona],[Milan],[Barcelona]
prompt_8_gpt-4o-mini,[Plovdiv],[Bergen],[]
prompt_9_gemini-1.5-pro-001,[Ankara],"[Budapest, Luxembourg, Mykolaiv, Nicosia, Orad...","[Amsterdam, Ankara, Budapest]"


In [28]:
pivot_table_sar = df.pivot_table(index='prompt_id', columns='model', values='rec_cities_sar', aggfunc=lambda x: x)
pivot_table_sar

model,gemma2,llama3point1-instruct,mistral-instruct
prompt_id,,,
prompt_0_gemini-1.5-pro-001,[Nalchik],"[Bergen, Innsbruck, Kars, Kaunas, Nalchik, Plo...","[Kars, Kaunas, Nalchik, Plovdiv]"
prompt_0_gemini-ui,[Nalchik],"[Innsbruck, Plovdiv, Stavanger]","[Bergen, Kars, Kaunas, Nalchik, Plovdiv]"
prompt_0_gpt-4o-mini,[Strasbourg],"[Kaunas, Oradea, Sibiu]",[Strasbourg]
prompt_10_gemini-1.5-pro-001,[Kazan],"[Erzurum, Sarajevo, Vitoria-Gasteiz]","[Aalborg, Kazan, London, Nalchik]"
prompt_10_gemini-ui,"[London, Mykolaiv]",[Varna],"[Mykolaiv, Simferopol]"
...,...,...,...
prompt_8_gemini-ui,"[Antalya, Barcelona]",[Milan],"[Barcelona, Braga, Milan, Tallinn]"
prompt_8_gpt-4o-mini,[Plovdiv],"[Arkhangelsk, Bergen, Varna]",[]
prompt_9_gemini-1.5-pro-001,[Ankara],[Budapest],"[Amsterdam, Ankara, Vienna]"


In [14]:
count_non_sus = count_same_response(pivot_table)

In [15]:
count_sar = count_same_response(pivot_table_sar)

In [16]:
print(count_non_sus, count_sar)

8 2


In [26]:
def count_atleast_one_common_city(df):
    count_same_values = 0

    for i, row in df.iterrows():
        l1 = set(row['llama3point1-instruct'])
        l2 = set(row['mistral-instruct'])
        l3 = set(row['gemma2'])

        if len(l1 & l2 & l3):
            count_same_values += 1


    # print("Count of records where values of C are the same across different values of A:\n", count_same_values)
    return count_same_values

In [30]:
one_common = count_atleast_one_common_city(pivot_table)
one_common_sar = count_atleast_one_common_city(pivot_table_sar)

print(one_common, one_common_sar)

27 29
